In [1]:
import numpy as np
import pandas as pd
import random
# from sqlalchemy.engine import create_engine
from sqlalchemy import create_engine

from mysql.connector import connect
import mysql.connector
from mysql.connector import Error
import pymysql
import warnings
import traceback
from pandas.io import sql
from sqlalchemy import create_engine

from random import randrange
import datetime
from datetime import timedelta
from sklearn import preprocessing
from sqlalchemy import exc

# import datetime

# import MySQLdb

warnings.filterwarnings("ignore")

In [2]:
from urllib.parse import quote_plus


In [3]:
# connect db
# mysqlsh -u doadmin -p -h tastie-do-user-11494132-0.b.db.ondigitalocean.com -P 25060 -D defaultdb

def connect_db():
    user_name = "doadmin"
    pwd = "AVNS_dIQZdxGNQnKqj_q" 
    server = "tastie-do-user-11494132-0.b.db.ondigitalocean.com"

    # server = 'private-tastie-do-user-11494132-0.b.db.ondigitalocean.com'
    # pwd = 'AVNS_dIQZdxGNQnKqj_q'
    db = "Tastie"
    conn = connect(host=server, user=user_name, password=pwd, database=db, port=25060)
    cur = conn.cursor(buffered=True)
    return conn, cur


# conn = create_engine(conn_str)

In [4]:
user_name = "doadmin"
pwd = "AVNS_dIQZdxGNQnKqj_q"
server = "tastie-do-user-11494132-0.b.db.ondigitalocean.com"


engine = create_engine(
    "mysql+pymysql://{user}:{pw}@{host}:{port}/{db}".format(
        user=user_name, pw=pwd, db="Tastie", host=server, port=25060
    )
)


In [5]:
# engine.connect()


In [6]:
# "mysql+pymysql://{user}:{pw}@{host}/{db}".format(
#     user=user_name, pw=pwd, db="Tastie", host=server
# )


In [7]:
def close_connection(cur, conn):
    cur.close()
    conn.close()

In [8]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


In [9]:
def insert_execute_many(dataset, table_name):
    conn, cur = connect_db()

    cols = "`,`".join([str(i) for i in list(dataset)])
    sql = (
        "INSERT INTO `"
        + table_name
        + "` (`"
        + cols
        + "`) VALUES ("
        + "%s," * (dataset.shape[1] - 1)
        + "%s)"
    )

    try:
        cur.executemany(sql, dataset.values.tolist())
        conn.commit()
    except Error as error:
        conn.rollback()
        print("Failed to insert into MySQL table {}".format(error))
    finally:
        cur.close()
        conn.close()

In [10]:
import time

In [11]:
# for dataset > 100.000 records and many columns (ex: product)
def insert_execute_many_large_dataset(dataset, table_name):
    dataset_size = len(dataset)
    step = 5000
    cols = "`,`".join([str(i) for i in list(dataset)])
    sql = (
        "INSERT INTO `"
        + table_name
        + "` (`"
        + cols
        + "`) VALUES ("
        + "%s," * (dataset.shape[1] - 1)
        + "%s)"
    )

    
    for start in range(0, len(dataset), step):
        conn, cur = connect_db()

        if dataset_size < step:
            step = abs(step-dataset_size)
            end = start + step
            insert_execute_many(dataset[start:end], table_name)
            break
        end = start + step
        insert_execute_many(dataset[start:end], table_name) 


        # data_table = dataset[start:end].values.tolist()    
        # try:
        #     cur.executemany(sql, data_table)
        #     conn.commit()
        # except Error as error:
        #     conn.rollback()
        #     print("Failed to insert into MySQL table {}".format(error))
        # finally:
        #     cur.close()
        #     conn.close()

            
        dataset_size -= step


### User

In [6]:
df_user = pd.read_csv("mysql_data/user.csv")

In [7]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39128 entries, 0 to 39127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        39128 non-null  int64  
 1   first_name     39128 non-null  object 
 2   last_name      39128 non-null  object 
 3   gender         39128 non-null  int64  
 4   birthday       39128 non-null  object 
 5   email          39128 non-null  object 
 6   phone          39128 non-null  int64  
 7   password       39128 non-null  object 
 8   role           39128 non-null  int64  
 9   user_token     39128 non-null  object 
 10  registered_at  39128 non-null  object 
 11  update_at      39128 non-null  object 
 12  delete_at      0 non-null      float64
 13  avatar         0 non-null      float64
dtypes: float64(2), int64(4), object(8)
memory usage: 4.2+ MB


In [8]:
df_user.delete_at = (
    df_user.delete_at.fillna(np.nan).replace([np.nan], [None])
)

df_user.avatar = (
    df_user.avatar.fillna(np.nan).replace([np.nan], [None])
)

In [31]:
df_user.head()

,user_id,first_name,last_name,gender,birthday,email,phone,password,role,user_token,registered_at,update_at,delete_at,avatar
0,1000001,Adaline,Rolofs,0,1986-04-11,Altstadt@hubspot.com,127223056,i87Z8UWYV0x,3,6Sd5oV14jY=4hGnBF19T698P45MJilxG#9I49MbgL8txjCU1,2019-12-11 19:47:00,2019-12-11 19:47:00,None,None
1,1000002,Lenny,Marquez,1,1970-01-06,Bounds@hubspot.com,814852386,zZS260t3a6oF&64wUJ855qLTe03819UpGTfM+Q7m,2,7VLM5aZU44H9#9eKRdf36985H9&V643g=1Ip5I62#D,2018-01-01 00:00:00,2018-01-01 00:00:00,None,None
2,1000003,Camara,Zywiyask,0,1988-08-17,Deimante.Leyendecker64769@yahoo.com,553575365,y5Y1EuCTa,3,p3hs5SfM2P35N1cPyoWT%3+gH5Gosczz4Cf046Ey0tO55N...,2018-01-01 01:53:00,2018-01-01 01:53:00,None,None
3,1000004,Seni,Besga,0,1975-01-12,MistiAThamry1672431299849@gmail.com,554173329,rxK%0v#1v178qsyzVNFRa5C2ar849MmPkBq4vu0FiSlU3u96,2,M2pJzT2+9758fj2twtO4s5x9p3j4N12K07mN3gZgSYH2j7...,2019-02-07 13:30:00,2019-02-07 13:30:00,None,None
4,1000005,Jaunita,Abdurakhmanoff,1,1975-01-07,AkilahOrta6@hubspot.com,980690649,QYG44f^nV67,1,4k97r92q270T9Pzkc0Fc4Il^MBb1MSC9,2022-01-09 05:12:00,2022-01-09 05:12:00,None,None


In [32]:
df_user.to_sql('User', con = engine, if_exists = 'append', chunksize = 10000, index= False)


39128

In [164]:
# insert_execute_many(df_user, 'User')

### Cart

In [33]:
df_cart = pd.read_csv("mysql_data/cart.csv")

In [34]:
df_cart.to_sql('Cart', con = engine, if_exists = 'append', chunksize = 10000, index= False)

13033

In [166]:
# insert_execute_many(df_cart, 'Cart')

### Owner

In [35]:
df_owner = pd.read_csv("mysql_data/owner.csv")

In [36]:
#df_owner.info()

df_owner.delete_at = (
    df_owner.delete_at.fillna(np.nan).replace([np.nan], [None])
)

In [37]:
df_owner.to_sql('Owner', con = engine, if_exists = 'append', chunksize = 10000, index= False)


12927

In [169]:
# insert_execute_many(df_owner, 'Owner')

### Provider

In [12]:
df_provider = pd.read_csv("mysql_data/provider.csv")

In [14]:
# df_provider

In [15]:
# df_provider.info()

df_provider.delete_at = (
    df_provider.delete_at.fillna(np.nan).replace([np.nan], [None])
)

df_provider.description = (
    df_provider.description.fillna(np.nan).replace([np.nan], [None])
)

In [16]:
df_provider[df_provider.provider_id == 1000074]

,provider_id,merchant_name,address,road,ward_id,district_id,city_id,description,keyword,hotline,...,status,erc,rush_hour,order_totals,user_id,owner_id,current_form,registered_at,update_at,delete_at
49,1000074,Stacked Pancake & Breakfast House,213,Trần Quý Khoách,1,2,3,None,abc,123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000153,1000049,5,2021-06-30 09:29:20,2021-06-30 09:29:20,None


In [17]:
try:
    df_provider.to_sql('Provider', con = engine, if_exists = 'append', chunksize = 1000, index= False)
except exc.IntegrityError as e:
    print(e)

OperationalError: (pymysql.err.OperationalError) (3730, "Cannot drop table 'Provider' referenced by a foreign key constraint 'FK_MenuCategory_belongto_Shop' on table 'MenuCategory'.")
[SQL: 
DROP TABLE `Provider`]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [41]:
# df_provider.to_sql('Provider', con = engine, if_exists = 'append', chunksize = 10000, index= False)


107

In [42]:
# insert_execute_many(df_provider, 'Provider')

### Provider Operation

In [43]:
df_provider_operation = pd.read_csv("mysql_data/provider_operation.csv")

In [44]:
#df_provider_operation.info()
df_provider_operation.delete_at = (
    df_provider_operation.delete_at.fillna(np.nan).replace([np.nan], [None])
)

In [45]:
df_provider_operation.to_sql('Operation', con = engine, if_exists = 'append', chunksize = 10000, index= False)


749

In [177]:
insert_execute_many_large_dataset(df_provider_operation, 'Operation')

### Product

In [54]:
df_product = pd.read_csv("mysql_data/product.csv")

In [55]:
# df_product.info()

df_product.delete_at = (
    df_product.delete_at.fillna(np.nan).replace([np.nan], [None])
)

df_product.description = (
    df_product.description.fillna(np.nan).replace([np.nan], [None])
)

df_product.product_image = (
    df_product.product_image.fillna(np.nan).replace([np.nan], [None])
)

In [56]:
try:
    df_product.to_sql('Product', con = engine, if_exists = 'append', chunksize = 10000, index= False)
except exc.IntegrityError as e:
    print(e)

In [57]:
# insert_execute_many_large_dataset(df_product, 'Product')

### Product Option

In [62]:
df_product_option = pd.read_csv("mysql_data/product_option.csv")

In [69]:
df_product_option.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12406 entries, 0 to 12405
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product_id          12406 non-null  int64 
 1   label               12406 non-null  object
 2   value               12406 non-null  object
 3   option_description  0 non-null      object
 4   is_required         12406 non-null  int64 
 5   price               12406 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 581.7+ KB


In [68]:
df_product_option['label'] = ''
df_product_option['value'] = ''
df_product_option['option_description'] = df_product_option['option_description'].fillna(np.nan).replace([np.nan], [None])

In [70]:
df_product_option.head()

,product_id,label,value,option_description,is_required,price
0,1000000,,,None,1,0
1,1000001,,,None,1,0
2,1000002,,,None,1,0
3,1000003,,,None,1,0
4,1000004,,,None,1,0


In [71]:
try:
    df_product_option.to_sql('ProductOption', con = engine, if_exists = 'append', chunksize = 10000, index= False)
except exc.IntegrityError as e:
    print(e)

In [72]:
# insert_execute_many_large_dataset(df_product_option, 'ProductOption')

### Provider Category

In [73]:
df_provider_category = pd.read_csv("mysql_data/provider_category.csv")

In [86]:
insert_execute_many(df_provider_category, 'ProviderCategory')

### Provider Category Detail

In [87]:
df_provider_category_detail = pd.read_csv("mysql_data/provider_category_detail.csv")

In [ ]:
# try:
#     df_provider_category_detail.to_sql('ProviderCategoryDetail', con = engine, if_exists = 'append', chunksize = 20000, index= False)
# except exc.IntegrityError as e:
#     print(e)

In [91]:
insert_execute_many_large_dataset(df_provider_category_detail, 'ProviderCategoryDetail')

### Cuisine Category

In [92]:
df_cuisine_category = pd.read_csv("mysql_data/cuisine_category.csv")

In [93]:
insert_execute_many_large_dataset(df_cuisine_category, 'CuisineCategory')

### Cuisine Category Detail

In [94]:
df_cuisine_category_detail = pd.read_csv("mysql_data/cuisine_category_detail.csv")

In [96]:
insert_execute_many_large_dataset(df_cuisine_category_detail, 'CuisineCategoryDetail')

### Main Food Category

In [97]:
df_main_food_category = pd.read_csv("mysql_data/main_food_category.csv")

In [99]:
insert_execute_many(df_main_food_category, 'MainFoodCategory')

### Main Food Category Detail

In [100]:
df_main_food_category_detail = pd.read_csv("mysql_data/main_food_category_detail.csv")

In [102]:
df_main_food_category_detail.shape

(107030, 2)

In [103]:
try:
    df_main_food_category_detail.to_sql(
        "MainFoodCategoryDetail",
        con=engine,
        if_exists="append",
        chunksize=50000,
        index=False,
    )
except exc.IntegrityError as e:
    print(e)


In [104]:
# insert_execute_many_large_dataset(df_main_food_category_detail, 'MainFoodCategoryDetail')

### Food Category

In [105]:
df_food_category = pd.read_csv("mysql_data/food_category.csv")

In [106]:
insert_execute_many(df_food_category, 'FoodCategory')

### Food Category Detail

In [107]:
df_food_category_detail = pd.read_csv("mysql_data/food_category_detail.csv")
df_food_category_detail.shape

(282721, 2)

In [108]:
try:
    df_food_category_detail.to_sql(
        "FoodCategoryDetail",
        con=engine,
        if_exists="append",
        chunksize=100000,
        index=False,
    )
except exc.IntegrityError as e:
    print(e)


In [109]:
# insert_execute_many_large_dataset(df_food_category_detail, 'FoodCategoryDetail')

### Menu Category

In [110]:
df_menu_category = pd.read_csv("mysql_data/menu_category.csv")

In [111]:
# df_menu_category.info()

In [112]:
insert_execute_many(df_menu_category, 'MenuCategory')

### Menu Category Detail

In [113]:
df_menu_category_detail = pd.read_csv("mysql_data/menu_category_detail.csv")
df_menu_category_detail.shape

(13325, 2)

In [ ]:
# try:
#     df_food_category_detail.to_sql(
#         "FoodCategoryDetail",
#         con=engine,
#         if_exists="append",
#         chunksize=100000,
#         index=False,
#     )
# except exc.IntegrityError as e:
#     print(e)


In [114]:
insert_execute_many_large_dataset(df_menu_category_detail, 'MenuCategoryDetail')

### **Shipper**

In [115]:
df_shipper = pd.read_csv('mysql_data/shipper.csv')

In [119]:
# df_shipper.info()

In [117]:
df_shipper['delete_at'] = df_shipper['delete_at'].fillna(np.nan).replace([np.nan], [None])


In [118]:
insert_execute_many(df_shipper, 'Shipper')

### **Order**

In [13]:
df_order = pd.read_csv("mysql_data/order.csv")

In [14]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499999 entries, 0 to 499998
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   order_id            499999 non-null  int64  
 1   customer_id         499999 non-null  int64  
 2   payment_id          499999 non-null  int64  
 3   shipper_id          499999 non-null  int64  
 4   order_code          499999 non-null  object 
 5   payment_status_id   499999 non-null  int64  
 6   promotion_id        0 non-null       float64
 7   ecoupon_id          0 non-null       float64
 8   customer_address    499999 non-null  object 
 9   customer_phone      499999 non-null  int64  
 10  delivery_method_id  499999 non-null  int64  
 11  delivery_fee        499999 non-null  float64
 12  tip                 499999 non-null  float64
 13  subtotal            499999 non-null  float64
 14  total_amount        499999 non-null  float64
 15  delivery_mode       499999 non-nul

In [15]:
df_order['promotion_id'] = df_order['promotion_id'].fillna(np.nan).replace([np.nan], [None])
df_order['ecoupon_id'] = df_order['ecoupon_id'].fillna(np.nan).replace([np.nan], [None])
df_order['subtotal'] = df_order['subtotal'].fillna(np.nan).replace([np.nan], [None])
df_order['delivery_mode'] = df_order['delivery_mode'].fillna(np.nan).replace([np.nan], [None])
df_order['schedule_time'] = df_order['schedule_time'].fillna(np.nan).replace([np.nan], [None])
df_order['total_amount'] = df_order['total_amount'].fillna(np.nan).replace([np.nan], [None])


In [16]:
try:
    df_order.to_sql(
        "Order",
        con=engine,
        if_exists="append",
        chunksize=20000,
        # method='multi',
        index=False,
    )
except exc.IntegrityError as e:
    print(e)


In [220]:
# insert_execute_many_large_dataset(df_order, "Order")

### **Order Detail**

In [12]:
df_order_detail = pd.read_csv("mysql_data/order_detail.csv")

In [15]:
df_order_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427308 entries, 0 to 1427307
Data columns (total 7 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   order_id             1427308 non-null  int64 
 1   product_id           1427308 non-null  int64 
 2   label                1427308 non-null  object
 3   value                1427308 non-null  object
 4   quantity             1427308 non-null  int64 
 5   special_instruction  633348 non-null   object
 6   item_code            1427308 non-null  object
dtypes: int64(3), object(4)
memory usage: 76.2+ MB


In [14]:
df_order_detail["special_instruction"] = (
    df_order_detail["special_instruction"].fillna(np.nan).replace([np.nan], [None])
)


In [21]:
try:
    df_order_detail.to_sql(
        "OrderDetail",
        con=engine,
        if_exists="append",
        chunksize=10000,
        method='multi',
        index=False,
    )
except exc.IntegrityError as e:
    print(e)


(pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`OrderDetail`, CONSTRAINT `FK_OrderDetail_Product` FOREIGN KEY (`product_id`) REFERENCES `Product` (`product_id`))')
[SQL: INSERT INTO `OrderDetail` (order_id, product_id, label, value, quantity, special_instruction, item_code) VALUES (%(order_id_m0)s, %(product_id_m0)s, %(label_m0)s, %(value_m0)s, %(quantity_m0)s, %(special_instruction_m0)s, %(item_code_m0)s), (%(order_id_m1)s, %(product_id_m1)s, %(label_m1)s, %(value_m1)s, %(quantity_m1)s, %(special_instruction_m1)s, %(item_code_m1)s), (%(order_id_m2)s, %(product_id_m2)s, %(label_m2)s, %(value_m2)s, %(quantity_m2)s, %(special_instruction_m2)s, %(item_code_m2)s), (%(order_id_m3)s, %(product_id_m3)s, %(label_m3)s, %(value_m3)s, %(quantity_m3)s, %(special_instruction_m3)s, %(item_code_m3)s), (%(order_id_m4)s, %(product_id_m4)s, %(label_m4)s, %(value_m4)s, %(quantity_m4)s, %(special_instruction_m4)s, %(item_code_m4)s), (%(order

In [22]:
# insert_execute_many_large_dataset(df_order_detail[:10000], "OrderDetail")

### **Order Status**

In [ ]:
df_order_status = pd.read_csv("mysql_data/order_status.csv")

In [ ]:
df_order_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499999 entries, 0 to 499998
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   order_id           499999 non-null  int64 
 1   update_at          499999 non-null  object
 2   order_status_name  499999 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 11.4+ MB


In [218]:
insert_execute_many_large_dataset(df_order_status, "OrderStatus")

Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`OrderStatus`, CONSTRAINT `FK_OrderStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`OrderStatus`, CONSTRAINT `FK_OrderStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`OrderStatus`, CONSTRAINT `FK_OrderStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`OrderStatus`, CONSTRAINT `FK_OrderStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (2300

### **Payment Status**

In [ ]:
df_payment_status = pd.read_csv("mysql_data/payment_status.csv")

In [ ]:
df_payment_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499999 entries, 0 to 499998
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   order_id   499999 non-null  int64 
 1   status     499999 non-null  int64 
 2   update_at  499999 non-null  object
dtypes: int64(2), object(1)
memory usage: 11.4+ MB


In [219]:
insert_execute_many_large_dataset(df_payment_status, "PaymentStatus")

Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`PaymentStatus`, CONSTRAINT `FK_PaymentStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`PaymentStatus`, CONSTRAINT `FK_PaymentStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`PaymentStatus`, CONSTRAINT `FK_PaymentStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Tastie`.`PaymentStatus`, CONSTRAINT `FK_PaymentStatus_belongTo_Order` FOREIGN KEY (`order_id`) REFERENCES `Order` (`order_id`))
Failed to insert into MySQL 